In [ ]:
data_one_id = data_prep.xs(226000188, level='WELL_ID')
data_one_id = data_one_id.drop(['targ_press_d','targ_press_h','targ_press_w'],axis=1)

In [ ]:
corr_matrix = data_one_id.corr()
high_corr = [
    column for column in lower.columns if any((lower[column] > 0.6)|(lower[column] < -0.6))
]

In [ ]:
other_features = [col for col in data_one_id.columns if col not in high_corr and col!='quality']


In [ ]:
data_features = data_one_id.copy()


In [ ]:
comparison_table, std_outliers = get_column_outliers(data_features)
anomalies_report(std_outliers)

In [ ]:
comparison_table, iqr_outliers = get_column_outliers(data_features, function=outlier_iqr)


In [ ]:
scaler = RobustScaler()
scaled_data = pd.DataFrame(
    data=scaler.fit_transform(data_features), 
    columns=data_features.columns
)

In [ ]:
euclidian_model = DistanceOutliers(metric='euclidean', percentile=90)
euclidian_model.fit(scaled_data)
euclidian_outliers = euclidian_model.predict(scaled_data)

In [ ]:
#DBSCAN
# для начала считаем все наблюдения аномальными
outlier_percentage = 1.

num_clusters = []
anomaly_percentage = []

# берем маленький эпсилон и начинаем увеличивать
eps = 0.05
eps_history = [eps]
while outlier_percentage>0.05:    
    model = DBSCAN(eps=eps).fit(scaled_data)
    labels = model.labels_
    num_clusters.append(len(np.unique(labels))-1)
    labels = np.array([1 if label == -1 else 0 for label in labels])
    # считаем текущий процент "шума"
    outlier_percentage = sum(labels==1) / len(labels)    
    eps += 0.05
    eps_history.append(eps)
    anomaly_percentage.append(outlier_percentage)
    
model = DBSCAN(eps)
model.fit(scaled_data)
density_outlier = np.array([1 if label == -1 else 0 for label in model.labels_])

model = DBSCAN(1.3)
model.fit(scaled_data)
density_outlier = np.array([1 if label == -1 else 0 for label in model.labels_])

In [ ]:
#SVM
one_class_svm = OneClassSVM(nu=0.1, gamma='auto')
one_class_svm.fit(scaled_data)
svm_outliers = one_class_svm.predict(scaled_data)
svm_outliers = np.array([1 if label == -1 else 0 for label in svm_outliers])

In [ ]:
#Iforest
isolation_forest = IsolationForest(n_estimators=100, contamination=0.1, 
                                   max_features=1.0, bootstrap=True, behaviour="new")
isolation_forest.fit(scaled_data)

isolation_outliers = isolation_forest.predict(scaled_data)
isolation_outliers = np.array([1 if label == -1 else 0 for label in isolation_outliers])

In [ ]:
summary = np.concatenate((
    [std_outliers], 
    [iqr_outliers], 
    [euclidian_outliers], 
#     [cityblock_outliers],
    [density_outlier],
    [svm_outliers],
    [isolation_outliers]
))

In [ ]:
summary = pd.DataFrame(
    summary.T, 
    columns=['std', 'iqr', 'euclid', 
#              'cityblock',
             'dbscan', 'svm', 'isolation']
)
summary.head()

In [ ]:
data_one_id_y = data_one_id_y.reset_index()
comparsion_with_y_outline = summary.copy()
comparsion_with_y_outline['2sigma'] = data_one_id_y['2sigma']
comparsion_with_y_outline['3sigma'] = data_one_id_y['3sigma']

comparsion_with_y_outline

In [ ]:
comparsion_with_y_outline = labeled_data['outlier_score']
comparsion_with_y_outline = pd.DataFrame(comparsion_with_y_outline)
comparsion_with_y_outline['2sigma'] = data_one_id_y['2sigma']
comparsion_with_y_outline['3sigma'] = data_one_id_y['3sigma']